# DS 440 Data Exploration

# Spring 2023
### Instructor: Prof. John Yen
### Group Name: Philly Boys
### Team Members: Andrew cavnaugh, Aman Arora


In [1]:
import pyspark
import csv


In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, LongType
from pyspark.sql.functions import col, column
from pyspark.sql.functions import expr
from pyspark.sql.functions import split
from pyspark.sql.functions import array_contains
from pyspark.sql import Row
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import PCA

In [ ]:
#Local mode
#Cluster mode
ss = SparkSession.builder.master("local").appName("Data Analysis Cusltering").getOrCreate()
#ss = SparkSession.builder.appName("Data Analysis Cusltering").getOrCreate()

In [4]:
#Read in the data
Scanners_df_D1 = ss.read.csv("/storage/home/aka5922/data/complete_Day1_2021profile.csv", header= True, inferSchema=True )
Scanners_df_D2 = ss.read.csv("/storage/home/aka5922/data/complete_Day2_2021profile.csv", header= True, inferSchema=True )

In [ ]:
# Need to filter the Extreme Scanners 

In [6]:
multi_port_scanners_D1= Scanners_df_D1.where(col("numports")>1)
multi_port_scanners_D2= Scanners_df_D2.where(col("numports")>1)

In [8]:
multi_port_scanners_count_D1 = multi_port_scanners_D1.count()
multi_port_scanners_count_D2 = multi_port_scanners_D2.count()

In [10]:
ScannersCount_byNumPorts_D1 = multi_port_scanners_D1.groupby("numports").count()

In [11]:
ScannersCount_byNumPorts_D2 = multi_port_scanners_D2.groupby("numports").count()

In [12]:
SortedNumPorts_DF_D1= ScannersCount_byNumPorts_D1.orderBy("numports", ascending = False)

In [13]:
SortedNumPorts_DF_D2= ScannersCount_byNumPorts_D2.orderBy("numports", ascending = False)

In [14]:
max_non_rare_NumPorts= 1000

In [15]:
extreme_scanners_D1 = Scanners_df_D1.where(col("numports") > max_non_rare_NumPorts)

In [16]:
extreme_scanners_D2 = Scanners_df_D2.where(col("numports") > max_non_rare_NumPorts)

In [17]:
#path2_D1="/storage/home/aka5922/Extreme_Scannersone.csv"
#extreme_scanners_D1.write.option("header",True).csv(path2_D1)

In [18]:
#path2_D2="/storage/home/aka5922/Extreme_Scannertwo.csv"
#extreme_scanners_D2.write.option("header",True).csv(path2_D2)

In [19]:
non_extreme_multi_port_scanners_D1 = Scanners_df_D1.where(col("numports") <= max_non_rare_NumPorts).where(col("numports") > 1)

In [20]:
non_extreme_multi_port_scanners_D2 = Scanners_df_D2.where(col("numports") <= max_non_rare_NumPorts).where(col("numports") > 1)

In [21]:
# Day1
NEMP_Scanners_df_D1=non_extreme_multi_port_scanners_D1.withColumn("Ports_Array", split(col("ports_scanned_str"), "-") )

In [22]:
# Day2
NEMP_Scanners_df_D2=non_extreme_multi_port_scanners_D2.withColumn("Ports_Array", split(col("ports_scanned_str"), "-") )

In [23]:
Ports_Scanned_RDD_D1 = NEMP_Scanners_df_D1.select("Ports_Array").rdd

In [24]:
Ports_Scanned_RDD_D2 = NEMP_Scanners_df_D2.select("Ports_Array").rdd

In [25]:
Ports_list_RDD_D1 = Ports_Scanned_RDD_D1.map(lambda row: row[0] )
Ports_list_RDD_D2 = Ports_Scanned_RDD_D2.map(lambda row: row[0] )

In [26]:
Ports_list2_RDD_D1 = Ports_list_RDD_D1.flatMap(lambda x: x )
Ports_list2_RDD_D2 = Ports_list_RDD_D2.flatMap(lambda x: x )

In [27]:
Port_count_RDD_D1 = Ports_list2_RDD_D1.map(lambda x: (x, 1))
Port_count_RDD_D2 = Ports_list2_RDD_D2.map(lambda x: (x, 1))

In [28]:
Port_count_total_RDD_D1 = Port_count_RDD_D1.reduceByKey(lambda x,y: x+y)
Port_count_total_RDD_D2 = Port_count_RDD_D2.reduceByKey(lambda x,y: x+y)

In [29]:
Port_count_total_RDD_D1.count()


65535

In [30]:
Port_count_total_RDD_D2.count()

65532

In [31]:
Sorted_Count_Port_RDD_D1 = Port_count_total_RDD_D1.map(lambda x: (x[1], x[0])).sortByKey( ascending = False)
Sorted_Count_Port_RDD_D2 = Port_count_total_RDD_D2.map(lambda x: (x[1], x[0])).sortByKey( ascending = False)

In [32]:
top_k_ports = 300
Sorted_Count_Port_RDD_D1.take(top_k_ports)

[(251924, '17136'),
 (251916, '17132'),
 (251531, '17128'),
 (251467, '17140'),
 (251276, '17138'),
 (250654, '17130'),
 (184218, '17142'),
 (184030, '17134'),
 (128832, '23'),
 (111837, '8080'),
 (111822, '80'),
 (49399, '8081'),
 (47547, '2323'),
 (37527, '81'),
 (33323, '49744'),
 (32398, '8443'),
 (31812, '34224'),
 (31715, '34216'),
 (31706, '34228'),
 (31676, '34218'),
 (31537, '33972'),
 (31512, '33964'),
 (31499, '33960'),
 (31493, '34226'),
 (31466, '34220'),
 (31420, '33968'),
 (31394, '33970'),
 (31321, '33962'),
 (30941, '49152'),
 (30171, '7574'),
 (27823, '5555'),
 (26391, '1023'),
 (25946, '52869'),
 (25174, '37215'),
 (25164, '56062'),
 (18810, '34230'),
 (18572, '34222'),
 (18570, '33966'),
 (18485, '33974'),
 (17410, '8181'),
 (17323, '60001'),
 (6782, '445'),
 (4979, '0'),
 (4170, '5353'),
 (4150, '6881'),
 (4027, '8000'),
 (3893, '11211'),
 (3834, '5060'),
 (3798, '8082'),
 (3755, '4000'),
 (3719, '8083'),
 (3653, '443'),
 (3290, '1433'),
 (3131, '30301'),
 (2659, '

In [33]:
Sorted_Count_Port_RDD_D2.take(top_k_ports)

[(256111, '17128'),
 (256071, '17138'),
 (255938, '17132'),
 (255841, '17136'),
 (255675, '17130'),
 (255267, '17140'),
 (187424, '17134'),
 (187143, '17142'),
 (115588, '8080'),
 (115503, '80'),
 (105589, '23'),
 (38254, '62904'),
 (37998, '81'),
 (32353, '34220'),
 (32325, '34226'),
 (32309, '34224'),
 (32293, '34216'),
 (32287, '34218'),
 (32231, '33970'),
 (32098, '33968'),
 (32036, '33960'),
 (32025, '33972'),
 (31993, '33964'),
 (31974, '34228'),
 (31884, '33962'),
 (31417, '8443'),
 (31349, '49152'),
 (30442, '7574'),
 (28626, '52475'),
 (26667, '5555'),
 (26164, '1023'),
 (26016, '2323'),
 (25187, '52869'),
 (24959, '37215'),
 (21922, '8081'),
 (19088, '34222'),
 (19052, '33974'),
 (19045, '34230'),
 (18669, '33966'),
 (17010, '8181'),
 (16917, '60001'),
 (9490, '8550'),
 (8476, '445'),
 (6870, '0'),
 (6401, '30740'),
 (5482, '22'),
 (4913, '17018'),
 (4900, '54046'),
 (4708, '43293'),
 (4307, '13599'),
 (4148, '57628'),
 (4127, '5407'),
 (4110, '64030'),
 (4084, '64284'),
 (40

In [34]:
Sorted_Ports_RDD_D1= Sorted_Count_Port_RDD_D1.map(lambda x: x[1] )
Top_Ports_list_D1 = Sorted_Ports_RDD_D1.take(top_k_ports)


In [35]:
Sorted_Ports_RDD_D2= Sorted_Count_Port_RDD_D2.map(lambda x: x[1] )
Top_Ports_list_D2 = Sorted_Ports_RDD_D2.take(top_k_ports)

In [36]:
Top_Ports_D1D2 = Top_Ports_list_D1
for i in range(0,199):
    same_port_found = False
    for j in range(0,199):
        if Top_Ports_D1D2[j] == Top_Ports_list_D2[i]:
            same_port_found = True
    if same_port_found == False:
        Top_Ports_D1D2.append(Top_Ports_list_D2[i])

In [37]:
Top_Ports_list_D1[299]


'55553'

In [38]:
Top_Ports_list_D2[299]

'2375'

In [39]:
FeatureName_D1 = "Port"+Top_Ports_list_D1[299]
FeatureName_D2 = "Port"+Top_Ports_list_D2[299]

In [40]:
FeatureName_D1

'Port55553'

In [41]:
FeatureName_D2

'Port2375'

In [42]:
from pyspark.sql.functions import array_contains

In [43]:
top_k_ports_D1D2= len(Top_Ports_D1D2)

In [44]:
# Initialize NEMP_Scanners2_df
NEMP_Scanners2_df_D1 = NEMP_Scanners_df_D1
for i in range(0, top_k_ports_D1D2):
    # "Port" + Top_Ports_list[i]  is the name of each new feature created through One Hot Encoding Top_Ports_list
    NEMP_Scanners3_df_D1 = NEMP_Scanners2_df_D1.withColumn("Port" + Top_Ports_D1D2[i], array_contains("Ports_Array", Top_Ports_D1D2[i]))
    NEMP_Scanners2_df_D1 = NEMP_Scanners3_df_D1

In [45]:
# Initialize NEMP_Scanners2_df
NEMP_Scanners2_df_D2 = NEMP_Scanners_df_D2
for i in range(0, top_k_ports_D1D2):
    # "Port" + Top_Ports_list[i]  is the name of each new feature created through One Hot Encoding Top_Ports_list
    NEMP_Scanners3_df_D2 = NEMP_Scanners2_df_D2.withColumn("Port" + Top_Ports_D1D2[i], array_contains("Ports_Array", Top_Ports_D1D2[i]))
    NEMP_Scanners2_df_D2 = NEMP_Scanners3_df_D2

In [46]:
input_features = [ ]
for i in range(0, top_k_ports_D1D2 ):
    input_features.append( "Port"+ Top_Ports_D1D2[i] )

In [47]:
va_D1 = VectorAssembler().setInputCols(input_features).setOutputCol("features")

In [48]:
va_D2 = VectorAssembler().setInputCols(input_features).setOutputCol("features")

In [49]:
data_D1= va_D1.transform(NEMP_Scanners2_df_D1)

In [50]:
data_D2= va_D2.transform(NEMP_Scanners2_df_D2)

In [51]:
total_clusters_D1D2 = top_k_ports_D1D2
km = KMeans(featuresCol= "features", predictionCol="prediction").setK(top_k_ports_D1D2).setSeed(123)
km.explainParams()

'distanceMeasure: the distance measure. Supported options: \'euclidean\' and \'cosine\'. (default: euclidean)\nfeaturesCol: features column name. (default: features, current: features)\ninitMode: The initialization algorithm. This can be either "random" to choose random points as initial cluster centers, or "k-means||" to use a parallel variant of k-means++ (default: k-means||)\ninitSteps: The number of steps for k-means|| initialization mode. Must be > 0. (default: 2)\nk: The number of clusters to create. Must be > 1. (default: 2, current: 372)\nmaxIter: max number of iterations (>= 0). (default: 20)\npredictionCol: prediction column name. (default: prediction, current: prediction)\nseed: random seed. (default: 1761262128906942653, current: 123)\ntol: the convergence tolerance for iterative algorithms (>= 0). (default: 0.0001)\nweightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)'

In [52]:
kmModel1_D1=km.fit(data_D1)

In [53]:
kmModel1_D2=km.fit(data_D2)

In [54]:
predictions_D1 = kmModel1_D1.transform(data_D1)

In [55]:
predictions_D2 = kmModel1_D2.transform(data_D2)

In [56]:
Cluster1_df_D1=predictions_D1.where(col("prediction")==0)

In [57]:
Cluster1_df_D2=predictions_D2.where(col("prediction")==0)

In [58]:
summary_D1 = kmModel1_D1.summary

In [59]:
summary_D2 = kmModel1_D2.summary

In [60]:
summary_D1.clusterSizes

[480,
 8255,
 3040,
 1185,
 1103,
 7077,
 491,
 717,
 4004,
 1533,
 24949,
 57225,
 2591,
 60680,
 918,
 784,
 8551,
 786,
 344,
 3551,
 226,
 428,
 1551,
 775,
 716,
 374,
 1593,
 1899,
 2062,
 570,
 2633,
 2613,
 440,
 5294,
 2344,
 6626,
 7497,
 436,
 2981,
 2914,
 250,
 2590,
 711,
 18737,
 498,
 1430,
 6537,
 1578,
 634,
 2486,
 322,
 369,
 523,
 2003,
 6530,
 148,
 721,
 607,
 369,
 470,
 1425,
 886,
 7254,
 1241,
 1048,
 452,
 169,
 198,
 604,
 1085,
 4317,
 3250,
 2706,
 2572,
 2364,
 2,
 3230,
 1281,
 898,
 3903,
 4036,
 969,
 2578,
 3630,
 408,
 4242,
 440,
 742,
 1868,
 479,
 582,
 8834,
 719,
 779,
 352,
 343,
 2233,
 572,
 1536,
 415,
 95,
 2311,
 1216,
 2035,
 4284,
 1792,
 386,
 341,
 1542,
 1955,
 279,
 9223,
 1405,
 6454,
 278,
 844,
 2075,
 1686,
 944,
 326,
 1267,
 1627,
 328,
 707,
 6142,
 1636,
 3346,
 329,
 333,
 1877,
 1489,
 322,
 3753,
 177,
 341,
 583,
 2004,
 2146,
 9532,
 452,
 469,
 4214,
 419,
 2540,
 6704,
 141,
 2176,
 1282,
 2295,
 6750,
 1989,
 6618,
 

In [61]:
summary_D2.clusterSizes

[2005,
 3177,
 6509,
 1642,
 3681,
 3358,
 9154,
 522,
 2763,
 1003,
 9553,
 59785,
 6608,
 28331,
 7275,
 2814,
 69898,
 321,
 2949,
 1154,
 2141,
 2084,
 477,
 756,
 525,
 580,
 857,
 1855,
 6625,
 491,
 463,
 369,
 844,
 6599,
 502,
 6686,
 5277,
 3121,
 448,
 512,
 403,
 1852,
 359,
 2524,
 6732,
 1341,
 8921,
 4597,
 493,
 1257,
 2511,
 1168,
 639,
 8010,
 4154,
 493,
 3422,
 870,
 213,
 508,
 634,
 1302,
 97,
 534,
 634,
 1393,
 477,
 962,
 469,
 467,
 2746,
 3504,
 3672,
 2572,
 1049,
 1615,
 427,
 1232,
 3775,
 1837,
 544,
 807,
 4146,
 1561,
 685,
 2525,
 436,
 1961,
 2736,
 519,
 636,
 3851,
 725,
 4596,
 1013,
 2451,
 829,
 2733,
 2556,
 1440,
 52,
 478,
 1529,
 410,
 5381,
 1218,
 350,
 486,
 2131,
 3339,
 755,
 7604,
 865,
 1337,
 2658,
 792,
 2701,
 826,
 3451,
 1361,
 90,
 2031,
 1874,
 449,
 1416,
 2381,
 1306,
 302,
 1280,
 932,
 1226,
 1006,
 2292,
 824,
 1385,
 6544,
 1516,
 1139,
 6748,
 216,
 397,
 3144,
 3742,
 2503,
 593,
 176,
 659,
 403,
 388,
 969,
 4816,
 129

In [62]:
evaluator_D1 = ClusteringEvaluator()
silhouette_D1 = evaluator_D1.evaluate(predictions_D1)

In [63]:
evaluator_D2 = ClusteringEvaluator()
silhouette_D2 = evaluator_D2.evaluate(predictions_D2)

In [64]:
centers_D1 = kmModel1_D1.clusterCenters()

In [65]:
centers_D2 = kmModel1_D2.clusterCenters()

In [66]:
import pandas as pd
import numpy as np
import math

In [67]:
# Define columns of the Pandas dataframe
column_list = ['cluster ID', 'size'] #mirai ratio
for feature in input_features:
    column_list.append(feature)
clusters_summary_df_D1 = pd.DataFrame( columns = column_list )
clusters_summary_df_D2 = pd.DataFrame( columns = column_list )

#day1
for i in range(0, total_clusters_D1D2):
    cluster_i = predictions_D1.where(col('prediction')==i)
    cluster_i_size = cluster_i.count()
    cluster_row = [i, cluster_i_size] #cluster_i_mirai_ratio
    for j in range(0, len(input_features)):
        cluster_row.append(centers_D1[i][j])
    clusters_summary_df_D1.loc[i]= cluster_row

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/storage/home/aka5922/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/storage/home/aka5922/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/aci/apps/anaconda3/2021.05_gcc-4.8.5/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
#day2
for i in range(0, total_clusters):
    cluster_i = predictions_D1.where(col('prediction')==i)
    cluster_i_size = cluster_i.count()
    cluster_row = [i, cluster_i_size]
    for j in range(0, len(input_features)):
        cluster_row.append(centers_D2[i][j])
    clusters_summary_df_D2.loc[i]= cluster_row

In [ ]:
path4_D1= "/storage/home/aka5922/Data clustered/Clusters_Summary_D1_2.csv"
clusters_summary_df_D1.to_csv(path4_D1, header=True)

In [ ]:
path4_D2= "/storage/home/aka5922/Data clustered/Clusters_Summary_D2_2.csv"
clusters_summary_df_D2.to_csv(path4_D2, header=True)

In [ ]:
J_Nearest_Clusters_D2_D1_df = pd.DataFrame(columns=["D2Cindex" , "D2Csize" , "D1NearestCindex","D1size","Similarity"])
centers_D2 = kmModel1_D2.clusterCenters()
centers_D1 = kmModel1_D1.clusterCenters()

i=0
for center2 in centers_D2
    j = 0
    highest_similarity =0
    closest_cluster = -1
    for center1 in centers_D1
        if np.sum(center1)!= 0:
            fuzzy_intersection = np.minimum(center1,center2)
            fuzzy_union = np.maximum(center1,center2)
            cluster_i_j_similarity = np.sum(fuzzy_intersection)/np.sum(fuzzy_union)
            if cluster_i_j_similarity > highest_similarity:
                highest_similarity = cluster_i_j_similarity
                closest_cluster = j
        j = j+1
    if closest_cluster != -1:
        J_Nearest_Clusters_D2_D1_df.loc[i]=[i, summary_D2.clusterSizes[i], closest_cluster, summary_d1.clusterSizes[closest_cluster],highest_similarity]
    else:
        J_Nearest_Clusters_D2_D1_df.loc[i]=[i,summary_D2.clusterSizes[i], -1, 0,0]
    i = i + 1

In [ ]:
path5= "/storage/home/aka5922/Clusters_Summary_J_Nearest_Clusters_D2_D1_df_1.csv"
clusters_summary_df.to_csv(path5, header=True)